***
## Хранение тестов в проекте

Как правило, тесты хранят в отдельных файлах, а эти файлы собирают в Python-пакеты: в реальном проекте может накопиться немало файлов с тестами — и будет удобно, если они собраны вместе. 

С точки зрения файловой структуры, **пакеты в Python** — это директории, содержащие файл `__init__.py`. Для пакетов Python создаёт отдельное пространство имён, что позволяет в разных директориях хранить файлы с одинаковым названием и обращаться к ним через точечную нотацию.

```
├── calc_code/
│   ├── __init__.py         
│   └── calculator.py  # Тестируемый код будет здесь.
└── tests/
    ├── __init__.py   
    └── test_calculator.py  # А тесты - здесь.
```

In [ ]:
# calculator.py
class MadCalculator:
    """Производит арифметические действия разной степени безумности."""

    def sum_string(self, first_num, second_num):
        """
        Складывает аргументы как строки и возвращает число,
        сформированное из них. Если один из аргументов меньше нуля,
        эмоционально отказывается работать.
        """
        if first_num < 0 or second_num < 0:
            raise ValueError('Я решительно отказываюсь работать!')
        return int(str(first_num) + str(second_num))

    def sum_args(self, *args):
        """Ожидает на вход числа. Возвращает сумму принятых аргументов."""
        return sum(args)

In [ ]:
# test_calculator.py
import unittest

# Импортируем класс MadCalculator.
from calc_code.calculator import MadCalculator


class TestCalc(unittest.TestCase):
    """Тестируем MadCalculator."""

    def test_sum_string(self):
        """Тестирование функции sum_string с конкатенацией строк."""
        # Создаём экземпляр класса MadCalculator.
        calc = MadCalculator()
        # Вызываем метод.
        act = calc.sum_string(1, 100500)
        # Сравниваем фактический результат с ожидаемым.
        self.assertEqual(act, 1100500, 'Метод sum_string работает неправильно.')

    def test_sum_string_negative_value(self):
        """Тестирование исключения с отрицательным числом."""
        # Создаём другой экземпляр класса MadCalculator.
        calc = MadCalculator()
        # Проверяем, что тест выдаст ошибку ValueError.
        with self.assertRaises(ValueError):
            # Вызываем метод.
            calc.sum_string(1, -100500)

    def test_sum_args(self):
        """Тестирование функции суммирования аргументов."""
        # Создаём ещё один экземпляр класса MadCalculator.
        calc = MadCalculator()
        # Вызываем метод.
        act = calc.sum_args(3, -3, 5)
        self.assertEqual(act, 5, 'Метод sum_args работает неправильно.')

***
## Паттерн тестирования AAA

Сейчас у всех тестов в файле *test_calculator.py* одинаковая структура: сначала создаётся объект калькулятора, потом выполняется определённое действие — вызов одного из методов калькулятора; в конце выполняется проверка. 

Эти три этапа можно определить как

1. подготовка,

2. действие,

3. проверка.

По-английски принято называть подобную структуру тестов **паттерном AAA: Arrange, Act, Assert**. 

**Arrange** (настройка) — в этом блоке кода подготавливаются какие-либо данные или необходимые условия для выполнения теста.

**Act** — выполнение или вызов тестируемого кода.

**Assert** — проверка, что выполненный код возвращает ожидаемый результат.

Согласно паттерну AAA в тесте может быть лишь один блок Arrange и один Assert. Если в одном тесте блоки Arrange и Assert повторяются — тест следует разбить как минимум на два.


In [ ]:
# Так не надо:
Тест 1:
Подготовка => Действие => Проверка => Другое действие => Другая проверка

# Лучше вот так:
Тест 1:
Подготовка => Действие => Проверка
Тест 2:
Подготовка => Другое действие => Другая проверка

***
## Фикстуры

При тестировании класса `MadCalculator` в каждом тесте создаётся экземпляр этого класса. Такой подход нормально работает, но нарушает принцип *DRY: Don’t repeat yourself!, «Не повторяйся!»*.

Вместо того чтобы в нескольких тестах несколько раз выполнять одни и те же действия, подготавливая данные для тестирования, лучше применить **фикстуры** — специальные методы и функции, которые выполняются перед запуском теста и подготавливают для него исходные данные.

В результате выполнения фикстур могут быть созданы любые данные, на которых будет проводиться тестирование:

* содержимое базы данных,

* набор файлов с необходимым содержанием,

* какие угодно программные объекты

* …и вообще всё, что требуется для тестов.

К фикстурам относят не только методы, подготавливающие данные для тестирования. После проведения тестов необходимо вернуть в исходное состояние среду, в которой проводилось тестирование (например, файловую структуру или набор переменных среды). 

Функции и методы, которые после тестирования удаляют всю информацию, созданную для проведения тестов, тоже относят к фикстурам. Таким образом, фикстуры — это все вспомогательные данные и функции, требующиеся как для подготовки тестов, так и для «уборки» после них. 

В unittest есть специальные инструменты для «уборки» после тестов:

* метод `tearDown`: вызывается после каждого теста;

* метод `tearDownClass`: вызывается один раз после запуска всех тестов класса.

***
## Метод setUp() — выполнение одинаковых задач перед каждым тестом

Для подготовки условий для тестов в библиотеке *unittest* можно применять метод `setUp()`. Он автоматически вызывается перед запуском каждого теста в классе. Это как раз то, что нужно на этапе подготовки к тестированию класса `MadCalculator`: для каждого из тестов необходимо создать экземпляр этого класса — вот пусть метод `setUp()` этим и занимается.


In [ ]:
# test_calculator.py
import unittest

from calc_code.calculator import MadCalculator


class TestCalc(unittest.TestCase):
    """Тестируем MadCalculator."""

    def setUp(self):
        """Подготовка прогона теста. Вызывается перед каждым тестом."""
        # Arrange - подготавливаем данные для каждого теста.
        self.calc = MadCalculator()

    def test_sum_string(self):
        """Тестирование функции sum_string с конкатенацией строк."""
        # Убираем создание calc, меняем calc на self.calc.
        act = self.calc.sum_string(1, 100500)
        self.assertEqual(act, 1100500, 'Метод sum_string работает неправильно.')
    
    def test_sum_string_negative_value(self):
        """Тестирование исключения с отрицательным числом."""
        with self.assertRaises(ValueError):
            # Убираем создание calc, меняем calc на self.calc.
            self.calc.sum_string(1, -100500)

    def test_sum_args(self):
        """Тестирование функции суммирования аргументов."""
        # Убираем создание calc, меняем calc на self.calc.
        act = self.calc.sum_args(3, -3, 5)
        self.assertEqual(act, 5, 'Метод sum_args работает неправильно.') 

Всё работает, как и раньше, но теперь код не дублируется, а чтобы изменить экземпляр  `self.calc` — будет достаточно изменить код в методе `setUp()` и не придётся вносить правки во все тесты.

Перед каждым тестом метод `setUp()` создаёт новый объект `calc`; в этом несложно убедиться: из метода `setUp()` напечатайте этот объект:

In [ ]:
...

    def setUp(self):
        """Подготовка прогона теста. Вызывается перед каждым тестом."""
        self.calc = MadCalculator()
        print(self.calc)  # Вывод объекта на печать.

... 

In [ ]:
Вывод в консоль будет примерно таким:
<calc_code.calculator.MadCalculator object at 0x000002B11A4AC790>
.<calc_code.calculator.MadCalculator object at 0x000002B11A4ACA30>
.<calc_code.calculator.MadCalculator object at 0x000002B11A4AC790>
.
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK 

Видно, что `print()` сработал три раза, и объект каждый раз занимал разные области в ячейках памяти. Это означает, что `setUp()` вызывается перед каждым тестом и заново создаёт объект `calc`.

***
## Метод setUpClass() — однократный запуск для всех тестов класса

Если в процессе тестов сам объект `calc` и его свойства не изменяются — нет необходимости создавать его заново перед каждым тестом; можно создать его один раз и проводить все тесты с одним экземпляром.  

Для подобных ситуаций используется метод `setUpClass()`; он вызывается один раз для всего класса. Обратите внимание: `setUpClass()` — это «метод класса», к нему обязательно нужно применить декоратор `@classmethod`; первый аргумент этого метода принято называть `cls`. 

Для вызова атрибута класса не требуется создавать объект: такой атрибут можно вызвать, обратившись напрямую к классу, например — `TestCalc.calc`. Но он также будет доступен и через обращение к объекту класса: `self.calc`. 

Измените название метода `setUp()` на `setUpClass()`, обозначьте его декоратором `@classmethod` и внесите в тело класса необходимые правки; строку с `print()` оставьте: посмотрим, как будет вызываться этот новый метод:

In [ ]:
# test_calculator.py
import unittest

from calc_code.calculator import MadCalculator


class TestCalc(unittest.TestCase):
    """Тестируем MadCalculator."""

    @classmethod  # Декорируем метод класса.
    def setUpClass(cls):
        """Вызывается один раз перед запуском всех тестов класса."""
        # Для создания объекта и обращения к нему вместо self применяем cls.
        cls.calc = MadCalculator()
        print(cls.calc)  # Обращаемся к объекту не self.calc, а cls.calc.

    def test_sum_string(self):
        """Тестирование функции sum_string с конкатенацией строк."""
        # Можно обращаться к объекту калькулятора через имя класса - TestCalc.calc
        act = TestCalc.calc.sum_string(1, 100500)
        self.assertEqual(act, 1100500, 'Метод sum_string работает неправильно.')

    def test_sum_string_second_negative_value(self):
        """Тестирование исключения с отрицательным числом."""
        with self.assertRaises(ValueError):
            # А можно обращаться к объекту калькулятора, как и раньше, 
            # через self: self.calc
            self.calc.sum_string(1, -100500)

    def test_sum_args(self):
        """Тестирование функции суммирования аргументов."""
        act = self.calc.sum_args(3, -3, 5)
        self.assertEqual(act, 5, 'Метод sum_args работает неправильно.') 

В консоли будет примерно такой вывод:


In [ ]:
<calc_code.calculator.MadCalculator object at 0x0000029F67320E50>
...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK 

В этот раз `print()` сработал только один раз — выполнение `setUpClass()` производится единожды.

***
## Другие примеры фикстур unittest

Помимо `setUp()` и `setUpClass()` в unittest используются и другие методы и функции для подготовки тестового окружения или «уборки» после тестов. 

In [1]:
import unittest


def setUpModule():
    """Вызывается один раз перед запуском любого класса из файла."""
    print('> setUpModule')


def tearDownModule():
    """Вызывается один раз после запуска любого класса из файла."""
    print('> tearDownModule')


class TestExample(unittest.TestCase):
    """Демонстрирует принцип работы тестов."""

    @classmethod
    def setUpClass(cls):
        """Вызывается один раз перед запуском всех тестов класса."""
        print('>> setUpClass')

    @classmethod
    def tearDownClass(cls):
        """Вызывается один раз после запуска всех тестов класса."""
        print('>> tearDownClass')

    def setUp(self):
        """Подготовка прогона теста. Вызывается перед каждым тестом."""
        print('>>> setUp')

    def tearDown(self):
        """Вызывается после каждого теста."""
        print('>>> tearDown')

    def test_one(self):  # Это имитация обычного теста.
        print('>>>> test_one')

    def test_one_more(self): # Имитация другого теста.
        print('>>>> test_one_more')


class YetAnotherTestExample(unittest.TestCase):
    """В этом тестовом классе нет никаких фикстур."""

    def test_without_class_fixtures(self):
        print('>>>> test_without_class_fixtures') 

Выполните тесты из этого файла. В консоли отобразятся строки, показывающие порядок выполнения встроенных методов и функций *unittest*.

Должен был получиться такой вывод:

In [ ]:
> setUpModule
>> setUpClass
>>> setUp
>>>> test_one
>>> tearDown
.>>> setUp
>>>> test_one_more
>>> tearDown
.>> tearDownClass
>>>> test_without_class_fixtures
.> tearDownModule

----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK 

Обратите внимание, что в тестах класса `YetAnotherTestExample` нет фикстур на уровне класса, но при этом функция `tearDownModule` всё равно выполняется только после того, как завершатся все тесты изо всех классов.

Названия `setUp`, `tearDownClass` и других встроенных методов unittest непривычны глазу (в Python, согласно требованиям PEP8, методам и функциям нужно давать названия в стиле *snake_case*, а не в стиле *camelCase*), но такое написание сложилось в силу исторических причин и при работе с этой библиотекой допускается.